# Health LLM Chatbot using Mistral-7B-Instruct

## Objective
Build a chatbot that answers general health-related questions using a Large Language Model (LLM), while implementing safety mechanisms to prevent harmful medical advice.

## Goals
- Provide general health information
- Avoid diagnosis and prescriptions
- Implement safety filtering
- Evaluate chatbot responses

## Model Selection

Model Used: Mistral-7B-Instruct  
Platform: Hugging Face Inference API  

Reason:
- Open-source model
- No GPU required (API-based)
- Free-tier access
- Suitable for instruction-following tasks

## Configure API Endpoint and Load API Token

In [54]:
import os
import requests

API_URL = "https://openrouter.ai/api/v1/chat/completions"

HF_TOKENN = os.getenv("HF_TOKENN")

## Define System Prompt

In [55]:
SYSTEM_PROMPT = """
You are a helpful medical assistant.
Provide general health information only.
Do not provide diagnosis, prescriptions, or exact dosages.
Encourage consulting a healthcare professional for serious concerns.
Keep responses clear and friendly.
"""

In [56]:
def is_safe_query(user_input):
    danger_keywords = [
        "suicide",
        "kill myself",
        "overdose",
        "exact dosage",
        "emergency treatment",
        "prescription amount"
    ]
    return not any(word in user_input.lower() for word in danger_keywords)

## Model Interaction Function to Call the LLM

In [57]:
def ask_model(user_input):
    if not HF_TOKENN:
        return "Error: API token not found. Check your environment variable."

    headers = {
    "Authorization": f"Bearer {HF_TOKENN}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost",
    "X-Title": "health-llm-chatbot"
}

    data = {
        "model": "mistralai/mistral-7b-instruct",
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_input}
        ],
        "max_tokens": 200
    }

    try:
        response = requests.post(API_URL, headers=headers, json=data, timeout=60)

        print("Status Code:", response.status_code)   
        print("Raw Response:", response.text)        

        response.raise_for_status()

    except requests.exceptions.HTTPError as e:
        return f"HTTP Error: {e}\n{response.text}"
    except requests.exceptions.RequestException as e:
        return f"Request Error: {e}"

    result = response.json()

    if "choices" in result and len(result["choices"]) > 0:
        return result["choices"][0]["message"]["content"]
    else:
        return f"Unexpected response format: {result}"


 ## Test the Chatbot

In [59]:
queries = [
    "What causes a sore throat?",
    "Is paracetamol safe for children?",
]

for q in queries:
    print("Q:", q)

    if is_safe_query(q):
        print("A:", ask_model(q))
    else:
        print("Blocked: Unsafe query detected.")

    print("-" * 50)

Q: What causes a sore throat?
Status Code: 200
Raw Response: 
         

         

         

         

         

         

         

         

         

         
{"id":"gen-1772046249-mxa8gYRWhe4zfB68K8yu","provider":"Together","model":"mistralai/mistral-7b-instruct","object":"chat.completion","created":1772046249,"choices":[{"logprobs":null,"finish_reason":"length","native_finish_reason":"length","index":0,"message":{"role":"assistant","content":"A sore throat is usually caused by irritation or inflammation in the throat, often due to:\n\n- **Viral infections** (most common, like colds, flu, or COVID-19)\n- **Bacterial infections** (less common, such as strep throat)\n- **Dry air or allergies** (dust, pollen, or dryness can irritate the throat)\n- **Straining your voice** (shouting, singing, or talking too much)\n- **Environmental irritants** (smoke, pollution, or chemical fumes)\n- **Acid reflux or heartburn** (stomach acid can travel up and irritate the throat)\n- **Dry cou